In [ ]:
import pandas as pd
import spacy
import re
import seaborn as sns
sns.set()
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
from IPython.display import IFrame
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.cluster import AffinityPropagation
from scipy.spatial import distance
import gensim
import gensim.downloader as api
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from collections import Counter
import numpy as np

import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt', quiet=True)
nltk.download('words')
from nltk import stem
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk import word_tokenize
import matplotlib.pyplot as plt
from collections import defaultdict
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
import string
punct = list(string.punctuation)
import ast
from nltk.sentiment import SentimentIntensityAnalyzer

from sklearn.metrics import mean_squared_error, r2_score
from sklearn import datasets
import plotly.graph_objects as go
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import r_regression
from sklearn.model_selection import train_test_split
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
from IPython.display import IFrame
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.cluster import AffinityPropagation
from scipy.spatial import distance
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from gensim.models import KeyedVectors


import xgboost as xgb
import adjustText
from wordcloud import WordCloud
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize

# 1.0 Import Dataset

In [ ]:
df = pd.read_csv('1news.csv')

In [ ]:
df

In [ ]:
df.columns

In [ ]:
ex = pd.read_excel('1news.xlsx')

In [ ]:
ex

In [ ]:
ex.columns

# 3.0 Topic Modeling

In [ ]:
 vectorizer = TfidfVectorizer(input = 'content', strip_accents = 'ascii', 
                              stop_words = 'english')

In [ ]:
data = ex

data = data.drop_duplicates(subset = ['headline']).reset_index(drop = True)

data = data.sample(500).reset_index(drop = True)

In [ ]:
data

In [ ]:
text = [i for i in data['headline']]

In [ ]:
# Split text into sentences and remove trailing and leading whitespace
sentences = []
for sentence in text:
    sentences.extend(sentence.split('.'))
sentences = [s.strip() for s in sentences if s.strip()]

In [ ]:
text = []
for i in sentences:
    text.append(i.lower())

for i in range(len(sentences)):
    if isinstance(sentences[i], str):
        sentences[i] = sentences[i].lower()

In [ ]:
type(text[0])

In [ ]:
vectors = vectorizer.fit_transform(text)

In [ ]:
vectors = vectors.todense().tolist()

In [ ]:
ff = pd.DataFrame(
    vectors,columns=vectorizer.get_feature_names_out())

In [ ]:
ff

In [ ]:
pca_1 = PCA(n_components = 3)
comps_1 = pca_1.fit_transform(ff)
pc_ff_1 = pd.DataFrame(data = comps_1, columns = ['PC'+str(i) for i in range(1, comps_1.shape[1]+1)])

In [ ]:
pc_ff_1

In [ ]:
clustering = AgglomerativeClustering(n_clusters = 3, affinity = 'euclidean', linkage = 'ward').fit(ff)

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=0, n_init=10).fit(ff)

In [ ]:
clustering = AgglomerativeClustering(n_clusters = 3, affinity = 'euclidean', linkage = 'ward').fit(pc_ff_1)

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=0, n_init=10).fit(pc_ff_1)

In [ ]:
ff_all = pd.concat([ff, pc_ff_1], axis = 1)
ff_all['clusters_ag'] = [str(i) for i in clustering.labels_]
ff_all['clusters_knn'] = [str(i) for i in kmeans.labels_]
ff_all['headline'] = data['headline']
ff_all['topic'] = data['topic']

In [ ]:
fig = px.scatter_3d(ff_all, x='PC1', y='PC2', z='PC3',
              color='clusters_ag', hover_data = ['headline'])

fig.update_traces(marker=dict(size = 5, line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

fig.show('notebook')

# 4.0 Named Entity Recognition

In [ ]:
df

In [ ]:
nlp = spacy.load("en_core_web_lg")

entities = []
labels = []
position_start = []
position_end = []

for item in df['headline']: 
    doc = nlp(str(item)) 
    for ent in doc.ents:
        entities.append(ent)
        labels.append(ent.label_)
        position_start.append(ent.start_char)
        position_end.append(ent.end_char)
df_ner = pd.DataFrame({'Entities':entities, 'Labels':labels,'Position_Start': position_start, 'Position_end':position_end})

In [ ]:
df_ner

In [ ]:
df_ner['headline'] = df['headline']
label_counts = df_ner['Labels'].value_counts()
entity_freq = df_ner.groupby('Labels').size().reset_index(name='count')
# sort the entities by frequency in descending order
entity_freq = entity_freq.sort_values(by=['count'], ascending=False)
df_ner['headline'] = df['headline']
label_counts = df_ner['Labels'].value_counts()

In [ ]:
label_counts

In [ ]:
labels = ['ORG', 'DATE', 'NORP', 'PERSON', 'GPE', 'ORDINAL', 'CARDINAL', 'MONEY', 'LANGUAGE', 'TIME', 'QUANTITY', 'PERCENT']
counts = [536, 238, 104, 44, 37, 22, 15, 13, 5, 4, 1, 1]
plt.bar(labels, counts)
plt.xticks(rotation=90)
plt.xlabel('NER Label')
plt.ylabel('Count')
plt.title('Distribution of NER Labels')
plt.show()

In [ ]:
# Save figure as PNG
fig.write_image("bargraphNER.png")

In [ ]:
entity_freq

In [ ]:
total_count = sum(counts)

entity_freq = {'Labels': labels, 'count': counts}
percentages = [(count / total_count) * 100 for count in counts]

# Convert percentages to one-dimensional array
percentages = np.ravel(percentages)

# Create pie chart
plt.figure(figsize=(8,8))
plt.pie(percentages, labels=labels, autopct='%1.1f%%')
plt.title('Percentage Distribution of NER Labels')
plt.show()

# 5.0 Word Cloud

In [ ]:
df

In [ ]:
# Split text into sentences and remove trailing and leading whitespace
text = []
for sentence in df['headline']:
    text.extend(sentence.split('.'))
text = [s.strip() for s in text if s.strip()]

In [ ]:
# Collapse the list of strings into a single long string for processing
text = ' '.join(text)

In [ ]:
# Remove forward slashes ("/")
text = text.replace('/', '')

# Remove single quotes ("'")
text = text.replace("'", '')

# Remove square brackets ("'")
text = text.replace("[", '')

# Remove square brackets ("'")
text = text.replace("]", '')

In [ ]:
text

In [ ]:
# Get the list of English stop words
stop_words = set(stopwords.words('english'))

In [ ]:
word_tokens = word_tokenize(text)
# converts the words in word_tokens to lower case and then checks whether
#they are present in stop_words or not
filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
#with no lower case conversion
filtered_sentence = []
 
for w in word_tokens:
    if w not in stop_words:
        filtered_sentence.append(w)

In [ ]:
filtered_sentence

In [ ]:
# Remove commas (",") from the list of strings
stops = [s.replace(',', '') for s in filtered_sentence]

In [ ]:
# Remove empty strings from the list
stops = [s for s in stops if s]

In [ ]:
stops

In [ ]:
# Convert the list to a string
string = ' '.join(stops)

In [ ]:
# generate a wordcloud
word_cloud = WordCloud(collocations = False, background_color = 'white').generate(string)

In [ ]:
# print the wordcloud
plt.imshow(word_cloud, interpolation='bilinear')
plt.axis("off")
plt.show()